Import the libraries needed

In [1]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [2]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'


pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-05 21:57:29.657232
svmem(total=16756752384, available=9445961728, percent=43.6, used=7310790656, free=9445961728)


# 1. Get Broker and Commissions features.

  -  Identified in Lit Review.# 1. Read back the population with switches data.

In [3]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

broker_query = f"""SELECT pol.Policy_Number AS ContractNumber
	  ,pol.Servicing_Broker_Id AS ServicingBroker
      ,pol.Initial_Broker_Id   AS InitialBroker
      ,pb.Broker_Category_En AS BrokerCategory
	  ,AVG(cr.Initial_Rate) AS InitialRate
	  ,AVG(cr.renewal_Rate) AS RenewalRate
FROM cle_reporting.dbo.CLT_Policy pol
INNER JOIN cle_reporting.dbo.CLT_Product prd ON prd.Product_Id = pol.Product_Id
INNER JOIN  CLE_Reporting.dbo.CLT_Policy_Broker pb ON pb.Statement_Broker_Id = pol.Servicing_Broker_Id
INNER JOIN [CLE_Reporting].[dbo].[CLT_Commission_Rate] cr ON cr.Policy_Broker_Id = pb.Policy_Broker_Type_Id --AND cr.Policy_Broker_Id = pb.Policy_Broker_Id
WHERE prd.Product_Id = pol.Product_Id
AND   prd.Product_Code IN ('FKP_2006V1','FKP_2006V2','FKP_2008V1','FKP_2010V1','FKP_2012V1','FKP_2017V1',
                                            'GNPP_2016V1','GNPP_2017V1','GNPP_2022V1','GNPP_2023V1',
                                            'GNRP_2008V1','GNRP_2010V1','GNRP_2012V1','GNRP_2017V1','GNRP_2022V1',
                                            'XRPP_2005V1','XRPP_2008V1','XRPP_2010V1','XRPP_2012V1','XRPP_2015V1')
AND SUBSTRING(prd.Product_Category_Code,1,3) = SUBSTRING(cr.Commissions_Product_Group,1,3)
GROUP BY pol.Policy_Number,pol.Servicing_Broker_Id,pol.Initial_Broker_Id,pb.Broker_Category_En;"""

df_broker = pd.read_sql(broker_query, conn)

# Types and row count
# print stats.
print(len(df_broker.index))
now = datetime.now()
print(now)
print('==========================')


print(df_broker.dtypes)

# No displays to protect PII data
#df_broker.head(5)

Error: ('HY000', '[HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976) (SQLDriverConnect); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]SQL Server Network Interfaces: The system cannot contact a domain controller to service the authentication request. Please try again later.\r\n (-2146892976); [HY000] [Microsoft][ODBC Driver 17 for SQL Server]Cannot generate SSPI context (-2146892976)')

## Drop obsolete or repeated features.

## Check for nulls

In [4]:
df_broker.dtypes

NameError: name 'df_broker' is not defined

In [ ]:
df_broker.isna().any()

## All features well distributed so write to csv.

In [5]:
# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Broker_and_Commission_data.csv'
df_broker.to_csv(filename, encoding='utf-8', index=False)        

# No displays to protect PII data
#df_broker.head(10)

NameError: name 'df_broker' is not defined

In [6]:
%who_ls DataFrame 

[]

In [7]:
del df_broker





NameError: name 'df_broker' is not defined

In [8]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-05 21:58:54.331019
svmem(total=16756752384, available=9396920320, percent=43.9, used=7359832064, free=9396920320)
